In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV,RandomizedSearchCV
from sklearn.metrics import recall_score,precision_score,f1_score,accuracy_score, confusion_matrix, classification_report

from sklearn.ensemble import RandomForestClassifier
from explainerdashboard import ClassifierExplainer, ExplainerDashboard
from sklearn.tree import DecisionTreeClassifier

c:\users\monis\anaconda3\envs\ml\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\users\monis\anaconda3\envs\ml\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
c:\users\monis\anaconda3\envs\ml\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:\n%s" %


In [2]:
df = pd.read_csv(r'C:\\Users\\monis\\Desktop\\POML\\HICSP\\train.csv')
df.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,1,Male,44,1,28.0,0,> 2 Years,Yes,40454.0,26.0,217,1
1,2,Male,76,1,3.0,0,1-2 Year,No,33536.0,26.0,183,0
2,3,Male,47,1,28.0,0,> 2 Years,Yes,38294.0,26.0,27,1
3,4,Male,21,1,11.0,1,< 1 Year,No,28619.0,152.0,203,0
4,5,Female,29,1,41.0,1,< 1 Year,No,27496.0,152.0,39,0


In [3]:
df.drop(columns = 'id', inplace = True)
df.head()

# id wont be needed in the analysis and it wont be needed for the machine learning process so it's kind of redundant to keep it

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,Male,44,1,28.0,0,> 2 Years,Yes,40454.0,26.0,217,1
1,Male,76,1,3.0,0,1-2 Year,No,33536.0,26.0,183,0
2,Male,47,1,28.0,0,> 2 Years,Yes,38294.0,26.0,27,1
3,Male,21,1,11.0,1,< 1 Year,No,28619.0,152.0,203,0
4,Female,29,1,41.0,1,< 1 Year,No,27496.0,152.0,39,0


In [4]:
df['Gender'] = df['Gender'].map({'Female':1, 'Male':0})
df.head()

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,0,44,1,28.0,0,> 2 Years,Yes,40454.0,26.0,217,1
1,0,76,1,3.0,0,1-2 Year,No,33536.0,26.0,183,0
2,0,47,1,28.0,0,> 2 Years,Yes,38294.0,26.0,27,1
3,0,21,1,11.0,1,< 1 Year,No,28619.0,152.0,203,0
4,1,29,1,41.0,1,< 1 Year,No,27496.0,152.0,39,0


In [5]:
df['Vehicle_Damage'] = df['Vehicle_Damage'].map({'Yes':1, 'No':0})
df.head()


,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,0,44,1,28.0,0,> 2 Years,1,40454.0,26.0,217,1
1,0,76,1,3.0,0,1-2 Year,0,33536.0,26.0,183,0
2,0,47,1,28.0,0,> 2 Years,1,38294.0,26.0,27,1
3,0,21,1,11.0,1,< 1 Year,0,28619.0,152.0,203,0
4,1,29,1,41.0,1,< 1 Year,0,27496.0,152.0,39,0


In [6]:
df['Vehicle_Age'] = df['Vehicle_Age'].map({'1-2 Year':1, '< 1 Year':0, '> 2 Years': 2})
df.head()

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,0,44,1,28.0,0,2,1,40454.0,26.0,217,1
1,0,76,1,3.0,0,1,0,33536.0,26.0,183,0
2,0,47,1,28.0,0,2,1,38294.0,26.0,27,1
3,0,21,1,11.0,1,0,0,28619.0,152.0,203,0
4,1,29,1,41.0,1,0,0,27496.0,152.0,39,0


In [7]:
df.info()

# Now all column are in int or float value and ready to be machine learning processed

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381109 entries, 0 to 381108
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Gender                381109 non-null  int64  
 1   Age                   381109 non-null  int64  
 2   Driving_License       381109 non-null  int64  
 3   Region_Code           381109 non-null  float64
 4   Previously_Insured    381109 non-null  int64  
 5   Vehicle_Age           381109 non-null  int64  
 6   Vehicle_Damage        381109 non-null  int64  
 7   Annual_Premium        381109 non-null  float64
 8   Policy_Sales_Channel  381109 non-null  float64
 9   Vintage               381109 non-null  int64  
 10  Response              381109 non-null  int64  
dtypes: float64(3), int64(8)
memory usage: 32.0 MB


In [8]:
X = df.drop(columns = [ 'Driving_License','Response', 'Region_Code', 'Policy_Sales_Channel', 'Gender', 'Vintage'])
y = df['Response']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2)

In [10]:
import imblearn
from imblearn.over_sampling import SMOTE

In [11]:
sm = SMOTE(random_state = 42)

In [12]:
X_train.head()

,Age,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium
332803,39,0,1,1,52906.0
116248,38,0,1,1,23038.0
255005,22,1,0,0,45318.0
317474,23,1,0,0,29132.0
344212,56,0,2,1,2630.0


In [13]:
X_train_sm, y_train_sm = sm.fit_resample(X_train, y_train)

In [14]:
df_smote = pd.concat([X_train_sm, y_train_sm], axis = 1)

In [15]:
df_smote['Response'].value_counts()

0    267700
1    267700
Name: Response, dtype: int64

In [16]:
modelSMOTERF = RandomForestClassifier()

In [17]:
modelSMOTERF.fit(X_train_sm, y_train_sm)

RandomForestClassifier()

In [18]:
y_pred_SMOTE_RF = modelSMOTERF.predict(X_test)

In [19]:
RF_tuning = RandomForestClassifier()
param_DT =  {
    "n_estimators":[100,500,1000],
    "max_depth": [None, 4,6,8],
    "min_samples_leaf": [1,0.06,3,5],
    "min_samples_split" : [2,9,15,25],
    "max_features" : ['auto','sqrt', 'log2'],
    "criterion": ['gini', 'entropy']}

In [20]:
model_RF_tuned = RandomizedSearchCV(estimator=RF_tuning, param_distributions=param_DT,scoring = 'recall', verbose = 1, n_jobs = -1,cv = 3)

In [21]:
model_RF_tuned.fit(X_train_sm,y_train_sm)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=-1,
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [None, 4, 6, 8],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [1, 0.06, 3, 5],
                                        'min_samples_split': [2, 9, 15, 25],
                                        'n_estimators': [100, 500, 1000]},
                   scoring='recall', verbose=1)

In [22]:
RF_tuned = model_RF_tuned.best_estimator_

model_RF_tuned.best_estimator_

RandomForestClassifier(max_depth=6, min_samples_leaf=0.06, min_samples_split=25,
                       n_estimators=500)

In [23]:
y_tuned_RF = RF_tuned.predict(X_test)

In [24]:
cm_RF_tuned = confusion_matrix(y_test, y_tuned_RF, labels = [1,0])
cm_RF_tuned

array([[ 8747,   776],
       [22596, 44103]], dtype=int64)

In [25]:
acc_RF_tuned = accuracy_score(y_test, y_tuned_RF)
prec_RF_tuned = precision_score(y_test, y_tuned_RF)
rec_RF_tuned = recall_score(y_test, y_tuned_RF)
f1_RF_tuned = f1_score(y_test,y_tuned_RF)

print(classification_report(y_test, y_tuned_RF))

              precision    recall  f1-score   support

           0       0.98      0.66      0.79     66699
           1       0.28      0.92      0.43      9523

    accuracy                           0.69     76222
   macro avg       0.63      0.79      0.61     76222
weighted avg       0.89      0.69      0.75     76222



In [26]:
df_RF_tuned = pd.DataFrame(data = cm_RF_tuned , index = ["Actual 1","Actual 0"], columns = ["Prediction 1", "Prediction 0"])
df_RF_tuned

,Prediction 1,Prediction 0
Actual 1,8747,776
Actual 0,22596,44103


In [27]:
feature_descriptions = {
    "Age": "Age of the person", "Previously_Insured":"Whether the person is previously insured or not", "Vehicle_Age":"How long the vehicle had been used", "Vehicle_Damage":"whether the vehicle is damaged or not ",
       "Annual_Premium":"whats the annual premium you're paying"
}

In [28]:
X_train_sm.index.name = 'ID'

In [29]:
X_test.index.name = 'ID'

In [30]:
X_test.index

Int64Index([200222,  49766, 172201, 160713,  53272, 372603, 216160,  59206,
             26462,  95043,
            ...
             44507, 270739, 175441, 313395, 161247, 258403, 234155,  24476,
             60423, 185839],
           dtype='int64', name='ID', length=76222)

In [31]:
explainer = ClassifierExplainer(RF_tuned, X_test, y_test, 
                                descriptions=feature_descriptions,
                                labels=['Response-0', 'Response-1'], 
                                idxs = X_test.index, 
                                index_name = "ID", 
                                target = "Response", 
                                )
db = ExplainerDashboard(explainer, 
                        title="HICSP Explainer", 
                        shap_interaction=False,
                        )
db.run(port=8000)

Detected RandomForestClassifier model: Changing class type to RandomForestClassifierExplainer...
Note: model_output=='probability', so assuming that raw shap output of RandomForestClassifier is in probability space...
Generating self.shap_explainer = shap.TreeExplainer(model)
Building ExplainerDashboard..
Detected notebook environment, consider setting mode='external', mode='inline' or mode='jupyterlab' to keep the notebook interactive while the dashboard is running...
Generating layout...
Calculating shap values...
Calculating dependencies...
Calculating permutation importances (if slow, try setting n_jobs parameter)...
Calculating prediction probabilities...
Calculating classification_dfs...
Calculating pred_percentiles...
Calculating roc auc curves...
Calculating liftcurve_dfs...
Calculating pr auc curves...
Calculating confusion matrices...
Calculating predictions...
Calculating metrics...
Calculating ShadowDecTree for each individual decision tree...
Reminder: you can store the ex

 * Running on http://0.0.0.0:8000/ (Press CTRL+C to quit)
192.168.225.37 - - [04/May/2021 16:28:16] "GET / HTTP/1.1" 200 -
192.168.225.37 - - [04/May/2021 16:28:16] "GET /assets/bootstrap.min.css?m=1619528246.464637 HTTP/1.1" 200 -
192.168.225.37 - - [04/May/2021 16:28:16] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_9_1m1619528244.8.7.min.js HTTP/1.1" 200 -
192.168.225.37 - - [04/May/2021 16:28:16] "GET /_dash-component-suites/dash_renderer/react@16.v1_9_1m1619528244.14.0.min.js HTTP/1.1" 200 -
192.168.225.37 - - [04/May/2021 16:28:16] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_9_1m1619528244.7.2.min.js HTTP/1.1" 200 -
192.168.225.37 - - [04/May/2021 16:28:16] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_9_1m1619528244.14.0.min.js HTTP/1.1" 200 -
192.168.225.37 - - [04/May/2021 16:28:16] "GET /_dash-component-suites/dash_core_components/dash_core_components-shared.v1_16_0m1619528245.js HTTP/1.1" 200 -
192.168.225.37 - - [04/May/2021 16:28:16]

192.168.225.37 - - [04/May/2021 16:28:24] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.225.37 - - [04/May/2021 16:28:24] "POST /_dash-update-component HTTP/1.1" 204 -
192.168.225.37 - - [04/May/2021 16:28:24] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.225.37 - - [04/May/2021 16:28:24] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.225.37 - - [04/May/2021 16:28:25] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.225.37 - - [04/May/2021 16:28:25] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.225.37 - - [04/May/2021 16:28:25] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.225.37 - - [04/May/2021 16:28:25] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.225.37 - - [04/May/2021 16:28:25] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.225.37 - - [04/May/2021 16:28:25] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.225.37 - - [04/May/2021 16:28:25] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.225.37 - - [04/May/2021 

In [32]:
import pickle

Pkl_Filename = "Pickle_RF_tuned.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(RF_tuned, file)

In [33]:
Pkl_Filename = "Pickle_RF_tuned.pkl" 
with open(Pkl_Filename, 'rb') as file:  
    Pickled_RF_tuned_Model = pickle.load(file)

Pickled_RF_tuned_Model

RandomForestClassifier(max_depth=6, min_samples_leaf=0.06, min_samples_split=25,
                       n_estimators=500)